In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import csv
import random
import itertools
import sys
import shutil
from tqdm import tqdm
import helpermethods
from edgeml_tf.trainer.bonsaiTrainer import BonsaiTrainer
from edgeml_tf.graph.bonsai import Bonsai
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from scipy.stats import uniform
from data_utils import *
import re
import time
import pickle
import glob
from edgeml_tf.tflite.bonsaiLayer import BonsaiLayer
from tensorflow import keras
import tensorflow.compat.v1.keras.backend as K

Instructions for updating:
non-resource variables are not supported in the long term


## Import Dataset

In [2]:
f = '/home/nesl/209as_sec/human_act/Data/Activity_Dataset/'
model_dir = 'trained_models/'
window_size = 550
stride = 50
channels = 2

X_tr, Y_tr, X_test, Y_test = import_auritus_activity_dataset(dataset_folder = f, 
                                use_timestamp=False, 
                                shuffle=True, 
                                window_size = window_size, stride = stride, 
                                return_test_set = True, test_set_size = 300,channels=2)
print(X_tr.shape)
print(Y_tr.shape)
print(X_test.shape)
print(Y_test.shape)

100%|█████████████████████████████████████████| 399/399 [00:05<00:00, 74.90it/s]


(14671, 550, 2)
(14671, 9)
(300, 550, 2)
(300, 9)


# Feature Extraction

In [3]:
feat_size = 10
featX_tr = np.zeros((X_tr.shape[0],feat_size))
featX_test = np.zeros((X_test.shape[0],feat_size))
for i in range(X_tr.shape[0]):
    cur_win = X_tr[i]
    featX_tr[i,0] = np.min(cur_win[:,0])
    featX_tr[i,1] = np.min(cur_win[:,1])
    featX_tr[i,2] = np.max(cur_win[:,0])
    featX_tr[i,3] = np.max(cur_win[:,1])
    featX_tr[i,4] = featX_tr[i,2]-featX_tr[i,0]
    featX_tr[i,5] = featX_tr[i,3]-featX_tr[i,1]
    featX_tr[i,6] = np.var(cur_win[:,0])
    featX_tr[i,7] = np.var(cur_win[:,1])
    featX_tr[i,8] = np.sqrt(featX_tr[i,6])
    featX_tr[i,9] = np.sqrt(featX_tr[i,7])  
    
for i in range(X_test.shape[0]):
    cur_win = X_test[i]
    featX_test[i,0] = np.min(cur_win[:,0])
    featX_test[i,1] = np.min(cur_win[:,1])
    featX_test[i,2] = np.max(cur_win[:,0])
    featX_test[i,3] = np.max(cur_win[:,1])
    featX_test[i,4] = featX_test[i,2]-featX_test[i,0]
    featX_test[i,5] = featX_test[i,3]-featX_test[i,1]
    featX_test[i,6] = np.var(cur_win[:,0])
    featX_test[i,7] = np.var(cur_win[:,1])
    featX_test[i,8] = np.sqrt(featX_test[i,6])
    featX_test[i,9] = np.sqrt(featX_test[i,7])

dataDimension = featX_tr.shape[1]
numClasses = Y_tr.shape[1]
Xtrain = featX_tr
Ytrain = Y_tr
Xtest = featX_test
Ytest = Y_test

# Training and Hyperparameter Tuning


In [4]:
totalEpochs = 1000 #epochs to train each model for

In [5]:
depth = 3
projectionDimension = 22
sigma = 1.0

regZ = 0.0001
regW = 0.001
regV = 0.001
regT = 0.001

learningRate = 0.01
outFile = None
#Sparsity for Bonsai Parameters. x => 100*x % are non-zeros
sparZ = 0.2
sparW = 0.3
sparV = 0.3
sparT = 0.62
batchSize = np.maximum(100, int(np.ceil(np.sqrt(Ytrain.shape[0]))))
useMCHLoss = True #(False = Cross Entropy)
#Bonsai uses one classier for Binary, thus this condition
if numClasses == 2:
    numClasses = 1

X = tf.placeholder("float32", [None, dataDimension])
Y = tf.placeholder("float32", [None, numClasses])

dataDir = model_dir
shutil.rmtree(dataDir, ignore_errors=True)
os.mkdir(dataDir)
currDir = helpermethods.createTimeStampDir(dataDir)
helpermethods.dumpCommand(sys.argv, currDir)

#Instantiating the Bonsai Graph which will be used for training and inference.
bonsaiObj = Bonsai(numClasses, dataDimension, projectionDimension, depth, sigma)

#Instantiating the Bonsai Trainer which will be used for 3 phase training.
bonsaiTrainer = BonsaiTrainer(bonsaiObj, regW, regT, regV, regZ, sparW, sparT, sparV, sparZ,
                          learningRate, X, Y, useMCHLoss, outFile)
#Session declaration and variable initialization. Interactive Session doesn't clog the entire GPU.
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

#The method to to run the 3 phase training, followed by giving out the best early stopping model, accuracy along with saving of the parameters.
bonsaiTrainer.train(batchSize, totalEpochs, sess,
                Xtrain, Xtest, Ytrain, Ytest, dataDir, currDir)


Epoch Number: 0

******************** Dense Training Phase Started ********************


Classification Train Loss: 46.062028900782266
Training accuracy (Classification): 0.26871584399292864
Test accuracy 0.263333
MarginLoss + RegLoss: 43.231525 + 1.7560543 = 44.98758


Epoch Number: 1

Classification Train Loss: 11.614117157459258
Training accuracy (Classification): 0.32356557113428913
Test accuracy 0.253333
MarginLoss + RegLoss: 27.857296 + 1.6355295 = 29.492825


Epoch Number: 2

Classification Train Loss: 8.513335879643758
Training accuracy (Classification): 0.362226772805055
Test accuracy 0.286667
MarginLoss + RegLoss: 34.36389 + 1.583809 = 35.9477


Epoch Number: 3

Classification Train Loss: 8.717189554373423
Training accuracy (Classification): 0.36475409505267936
Test accuracy 0.223333
MarginLoss + RegLoss: 34.346478 + 1.5458817 = 35.89236


Epoch Number: 4

Classification Train Loss: 7.790257585048676
Training accuracy (Classification): 0.3759562820196152
Test accuracy 0.183


Epoch Number: 42

Classification Train Loss: 4.202482497692108
Training accuracy (Classification): 0.5162568300962448
Test accuracy 0.233333
MarginLoss + RegLoss: 41.85985 + 1.0383503 = 42.8982


Epoch Number: 43

Classification Train Loss: 3.5136563460032146
Training accuracy (Classification): 0.5273907085259756
Test accuracy 0.273333
MarginLoss + RegLoss: 37.19719 + 1.0302624 = 38.22745


Epoch Number: 44

Classification Train Loss: 3.16719411611557
Training accuracy (Classification): 0.541598358998696
Test accuracy 0.283333
MarginLoss + RegLoss: 35.183334 + 1.0192206 = 36.202557


Epoch Number: 45

Classification Train Loss: 4.147088046868642
Training accuracy (Classification): 0.5038251323004563
Test accuracy 0.213333
MarginLoss + RegLoss: 54.447453 + 1.0133709 = 55.460823


Epoch Number: 46

Classification Train Loss: 3.9085599621136984
Training accuracy (Classification): 0.5237021843592325
Test accuracy 0.23
MarginLoss + RegLoss: 47.254486 + 1.0077534 = 48.262238


Epoch Number:


Epoch Number: 84

Classification Train Loss: 2.5397174378236134
Training accuracy (Classification): 0.5633879750967026
Test accuracy 0.463333
MarginLoss + RegLoss: 3.533007 + 0.714053 = 4.24706


Epoch Number: 85

Classification Train Loss: 2.5941701203584673
Training accuracy (Classification): 0.5644808712104956
Test accuracy 0.483333
MarginLoss + RegLoss: 2.8926346 + 0.70696616 = 3.5996008


Epoch Number: 86

Classification Train Loss: 2.1724726140499113
Training accuracy (Classification): 0.5838114698727925
Test accuracy 0.47
MarginLoss + RegLoss: 3.8715825 + 0.6926051 = 4.5641875


Epoch Number: 87

Classification Train Loss: 2.240552810827891
Training accuracy (Classification): 0.5883879753450553
Test accuracy 0.38
MarginLoss + RegLoss: 3.8392396 + 0.68549824 = 4.524738


Epoch Number: 88

Classification Train Loss: 2.147713306546211
Training accuracy (Classification): 0.5899590137104194
Test accuracy 0.453333
MarginLoss + RegLoss: 3.963505 + 0.672702 = 4.636207


Epoch Number: 8


Epoch Number: 126

Classification Train Loss: 2.2004600783189137
Training accuracy (Classification): 0.5579918012022972
Test accuracy 0.483333
MarginLoss + RegLoss: 1.8631167 + 0.37743717 = 2.2405539


Epoch Number: 127

Classification Train Loss: 1.4286625881989796
Training accuracy (Classification): 0.5988387947281202
Test accuracy 0.546667
MarginLoss + RegLoss: 1.4119055 + 0.3643498 = 1.7762554


Epoch Number: 128

Classification Train Loss: 1.4166486571232477
Training accuracy (Classification): 0.6020491761465867
Test accuracy 0.493333
MarginLoss + RegLoss: 1.5637558 + 0.35440624 = 1.918162


Epoch Number: 129

Classification Train Loss: 1.3181471024950346
Training accuracy (Classification): 0.6204918031891187
Test accuracy 0.543333
MarginLoss + RegLoss: 1.2893329 + 0.347542 = 1.6368749


Epoch Number: 130

Classification Train Loss: 1.3024961898724239
Training accuracy (Classification): 0.6231557354331017
Test accuracy 0.503333
MarginLoss + RegLoss: 1.5006245 + 0.3373505 = 1.8379


Epoch Number: 167

Classification Train Loss: 1.0858256489038467
Training accuracy (Classification): 0.6178278649846712
Test accuracy 0.656667
MarginLoss + RegLoss: 0.770025 + 0.20367363 = 0.9736986


Epoch Number: 168

Classification Train Loss: 0.9918017307917277
Training accuracy (Classification): 0.6426229476928711
Test accuracy 0.586667
MarginLoss + RegLoss: 0.86104107 + 0.1982107 = 1.0592518


Epoch Number: 169

Classification Train Loss: 1.0532812928160031
Training accuracy (Classification): 0.6262978106737137
Test accuracy 0.65
MarginLoss + RegLoss: 0.76981753 + 0.19669366 = 0.9665112


Epoch Number: 170

Classification Train Loss: 1.0301784972349803
Training accuracy (Classification): 0.6317622939745585
Test accuracy 0.633333
MarginLoss + RegLoss: 0.83575267 + 0.1934492 = 1.0292019


Epoch Number: 171

Classification Train Loss: 1.1973203808069228
Training accuracy (Classification): 0.6115437132616838
Test accuracy 0.56
MarginLoss + RegLoss: 1.0181237 + 0.19802542 = 1.2161492

...


Epoch Number: 823

Classification Train Loss: 0.6655104075868924
Training accuracy (Classification): 0.6933743158976237
Test accuracy 0.716667
MarginLoss + RegLoss: 0.62572676 + 0.062965535 = 0.6886923


Epoch Number: 824

Classification Train Loss: 0.6668331479032834
Training accuracy (Classification): 0.6911885231733322
Test accuracy 0.7
MarginLoss + RegLoss: 0.60099643 + 0.06331228 = 0.6643087


Epoch Number: 825

Classification Train Loss: 0.667198832333088
Training accuracy (Classification): 0.6909152989586195
Test accuracy 0.703333
MarginLoss + RegLoss: 0.60299075 + 0.06327531 = 0.6662661


Epoch Number: 826

Classification Train Loss: 0.672312413652738
Training accuracy (Classification): 0.687704919775327
Test accuracy 0.67
MarginLoss + RegLoss: 0.6849695 + 0.06397588 = 0.74894536


Epoch Number: 827

Classification Train Loss: 0.7051147957642873
Training accuracy (Classification): 0.6801912570993106
Test accuracy 0.713333
MarginLoss + RegLoss: 0.60381114 + 0.06649769 = 0.67030


Epoch Number: 864

Classification Train Loss: 0.6651079967617989
Training accuracy (Classification): 0.6937158475319545
Test accuracy 0.706667
MarginLoss + RegLoss: 0.59733546 + 0.063684255 = 0.6610197


Epoch Number: 865

Classification Train Loss: 0.6695177291830381
Training accuracy (Classification): 0.6910519123077392
Test accuracy 0.706667
MarginLoss + RegLoss: 0.6163605 + 0.06410792 = 0.6804684


Epoch Number: 866

Classification Train Loss: 0.6665042077501615
Training accuracy (Classification): 0.6924863398075104
Test accuracy 0.703333
MarginLoss + RegLoss: 0.5845928 + 0.06458391 = 0.6491767


Epoch Number: 867

Classification Train Loss: 0.666189236442248
Training accuracy (Classification): 0.6937841544548671
Test accuracy 0.703333
MarginLoss + RegLoss: 0.60327375 + 0.06454712 = 0.6678209


Epoch Number: 868

Classification Train Loss: 0.675029709438483
Training accuracy (Classification): 0.6907786870996158
Test accuracy 0.71
MarginLoss + RegLoss: 0.5867724 + 0.065363444 = 0.6


Epoch Number: 905

Classification Train Loss: 0.6658360446492831
Training accuracy (Classification): 0.6939890702565511
Test accuracy 0.72
MarginLoss + RegLoss: 0.56301636 + 0.06641288 = 0.6294292


Epoch Number: 906

Classification Train Loss: 0.6717540164788564
Training accuracy (Classification): 0.6907103806734085
Test accuracy 0.713333
MarginLoss + RegLoss: 0.5711801 + 0.067090765 = 0.63827085


Epoch Number: 907

Classification Train Loss: 0.665554408232371
Training accuracy (Classification): 0.6950819651285808
Test accuracy 0.696667
MarginLoss + RegLoss: 0.5804841 + 0.06632022 = 0.64680433


Epoch Number: 908

Classification Train Loss: 0.6697823345661164
Training accuracy (Classification): 0.6938524593909582
Test accuracy 0.72
MarginLoss + RegLoss: 0.58142626 + 0.06614563 = 0.6475719


Epoch Number: 909

Classification Train Loss: 0.6647763301928838
Training accuracy (Classification): 0.6956284180283546
Test accuracy 0.72
MarginLoss + RegLoss: 0.573287 + 0.06546131 = 0.6387483



Epoch Number: 946

Classification Train Loss: 0.6688802475730579
Training accuracy (Classification): 0.6931010906895002
Test accuracy 0.72
MarginLoss + RegLoss: 0.5621965 + 0.069003105 = 0.6311996


Epoch Number: 947

Classification Train Loss: 0.6751454482475917
Training accuracy (Classification): 0.69002732137839
Test accuracy 0.71
MarginLoss + RegLoss: 0.57847 + 0.0695653 = 0.6480353


Epoch Number: 948

Classification Train Loss: 0.6739796464641888
Training accuracy (Classification): 0.6922814210255941
Test accuracy 0.72
MarginLoss + RegLoss: 0.5614748 + 0.07022128 = 0.6316961


Epoch Number: 949

Classification Train Loss: 0.6698834975560506
Training accuracy (Classification): 0.6899590164422988
Test accuracy 0.72
MarginLoss + RegLoss: 0.5786747 + 0.0694777 = 0.64815235


Epoch Number: 950

Classification Train Loss: 0.6717535798748334
Training accuracy (Classification): 0.6905737737814586
Test accuracy 0.686667
MarginLoss + RegLoss: 0.6045246 + 0.06942454 = 0.6739491


Epoch Num


Epoch Number: 987

Classification Train Loss: 0.6777122795581818
Training accuracy (Classification): 0.6888661200801531
Test accuracy 0.693333
MarginLoss + RegLoss: 0.5940556 + 0.07293017 = 0.66698575


Epoch Number: 988

Classification Train Loss: 0.676641583442688
Training accuracy (Classification): 0.6883879765868187
Test accuracy 0.7
MarginLoss + RegLoss: 0.57692057 + 0.07275971 = 0.64968026


Epoch Number: 989

Classification Train Loss: 0.6792367761333783
Training accuracy (Classification): 0.6868852431575457
Test accuracy 0.703333
MarginLoss + RegLoss: 0.5770666 + 0.07305052 = 0.6501171


Epoch Number: 990

Classification Train Loss: 0.6764796217282613
Training accuracy (Classification): 0.6900956268111865
Test accuracy 0.7
MarginLoss + RegLoss: 0.5643105 + 0.0726585 = 0.636969


Epoch Number: 991

Classification Train Loss: 0.6747575968503952
Training accuracy (Classification): 0.6898224045832951
Test accuracy 0.71
MarginLoss + RegLoss: 0.56822217 + 0.072470225 = 0.6406924


E

# Reloading Model

In [7]:
Z = np.load( glob.glob(model_dir + "/**/Z.npy", recursive = True)[0], allow_pickle=True )
W = np.load( glob.glob(model_dir + "/**/W.npy", recursive = True)[0], allow_pickle=True )
V = np.load( glob.glob(model_dir + "/**/V.npy", recursive = True)[0], allow_pickle=True )
T = np.load( glob.glob(model_dir + "/**/T.npy", recursive = True)[0], allow_pickle=True )
hyperparams = np.load(glob.glob(model_dir + "/**/hyperParam.npy", 
                            recursive = True)[0], allow_pickle=True ).item()

n_dim = hyperparams['dataDim']
projectionDimension = hyperparams['projDim']
numClasses = hyperparams['numClasses']
depth = hyperparams['depth']
sigma = hyperparams['sigma']

dense = BonsaiLayer( numClasses, n_dim, projectionDimension, depth, sigma )
model = keras.Sequential([
keras.layers.InputLayer(n_dim),
dense
])

dummy_tensor = tf.convert_to_tensor( np.zeros((1,n_dim), np.float32) )
out_tensor = model(dummy_tensor)
dense.set_weights( [Z, W, V, T] )
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bonsai_layer_1 (BonsaiLayer) (None,)                   6314      
Total params: 6,314
Trainable params: 6,314
Non-trainable params: 0
_________________________________________________________________
